In [1]:
import pika
import uuid

In [2]:
credentials = pika.PlainCredentials('myguest', 'myguestpwd')
#URL='localhost'
URL='PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com'
parameters =  pika.ConnectionParameters(URL, port=5672, credentials=credentials)

Below we will create a connection, channel, and queue

**queue_declare(queue, passive=False, durable=False, exclusive=False, auto_delete=False, arguments=None, callback=None)**<br>
<br>
Declare queue, create if needed. This method creates or checks a queue. When creating a new queue the client can specify various properties that control the durability of the queue and its contents, and the level of sharing for the queue.

**basic_consume(queue, on_message_callback, auto_ack=False, exclusive=False, consumer_tag=None, arguments=None, callback=None)**

Sends the AMQP 0-9-1 command Basic.Consume to the broker and binds messages for the consumer_tag to the consumer callback. If you do not pass in a consumer_tag, one will be automatically generated for you. Returns the consumer tag.

**queue (str)** – The queue to consume from. Use the empty string to specify the most recent server-named queue for this channel

**on_message_callback**: (callable) –
The function to call when consuming with the **signature on_message_callback(channel, method, properties, body)**, where

* **channel**: pika.Channel 

* **method**: pika.spec.Basic.Deliver 

* **properties**: pika.spec.BasicProperties 

* **body**: bytes


### Message properties

The AMQP 0-9-1 protocol predefines a set of 14 properties that go with a message. Most of the properties are rarely used, with the exception of the following:

* delivery_mode: Marks a message as persistent (with a value of 2) or transient (any other value). 
* content_type: Used to describe the mime-type of the encoding. For example for the often used JSON encoding it is a good practice to set this property to: application/json.
* reply_to: Commonly used to name a callback queue.
* correlation_id: Useful to correlate RPC responses with requests.

**See https://www.rabbitmq.com/tutorials/tutorial-six-python.html for details**

In [3]:
class FibonacciRpcClient(object):

    def __init__(self):
        #self.connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost'))

        self.connection = pika.BlockingConnection(parameters)
        self.channel = self.connection.channel()

        myqueue = self.channel.queue_declare('', exclusive=True)
        print("myqueue:", myqueue)
        
        self.callback_queue = myqueue.method.queue #The queue we just created

        print("myqueue.method:", myqueue.method)
        
        # Let us start waiting for messages (The results of our call). 
        #When we recevie a message, we will the on_response call back function
        self.channel.basic_consume(
            queue=self.callback_queue, #the queue we created above
            on_message_callback=self.on_response, #The callback function to run when we receive a message from the workers
            auto_ack=True)

    def on_response(self, ch, method, props, body):
        if self.corr_id == props.correlation_id: #Check if our correlation id is the same as we get in the message
            self.response = body

    def call(self, n):
        self.response = None
        self.corr_id = str(uuid.uuid4()) # This is where we create our unique correlation id
        print('corr_id:', self.corr_id)
        
        # Now we are ready to send our message to the workers
        self.channel.basic_publish(
            exchange='',
            routing_key='rpc_queue',
            properties=pika.BasicProperties(
                reply_to=self.callback_queue,
                correlation_id=self.corr_id,
            ),
            body=str(n))
        #Waiting for the results
        while self.response is None:
            self.connection.process_data_events()
        return int(self.response)

In [9]:
#This will create one channel and queue. All communications will be done on that queue using different correlation ids. 
fibonacci_rpc = FibonacciRpcClient()

myqueue: <METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=0', 'queue=amq.gen-k1gW8M51FPr2EZtTbcS9kg'])>"])>
myqueue.method: <Queue.DeclareOk(['consumer_count=0', 'message_count=0', 'queue=amq.gen-k1gW8M51FPr2EZtTbcS9kg'])>


In [5]:
print(" [x] Requesting fib(30)")
response = fibonacci_rpc.call(30)
print(" [.] Got %r" % response)

 [x] Requesting fib(30)
corr_id: 8ae873a0-ccad-4750-849e-423f2d4be8d8
 [.] Got 832040


In [6]:
print(" [x] Requesting fib(20)")
response = fibonacci_rpc.call(20)
print(" [.] Got %r" % response)

 [x] Requesting fib(20)
corr_id: 26eba689-fa6e-42a8-85bf-575daaa50259
 [.] Got 6765


In [7]:
print(" [x] Requesting fib(32)")
response = fibonacci_rpc.call(32)
print(" [.] Got %r" % response)

 [x] Requesting fib(32)
corr_id: 37cf2fb2-4392-4f80-b1bb-25b86a476d01
 [.] Got 2178309


In [8]:
print(" [x] Requesting fib(14)")
response = fibonacci_rpc.call(14)
print(" [.] Got %r" % response)

 [x] Requesting fib(14)
corr_id: d06ca5f2-1179-449b-a5e4-480813470612
 [.] Got 377
